In [1]:
from utils.data_depth import DepthDataset
from utils.plot import plot_3d_pointcloud
import numpy as np

In [2]:
dataset = DepthDataset(partition='train', 
                       outputs=['depth', 'edge_label', 'edge_index'],
                       pc_mean=0.5,          # Hace que la media de valores de profundidad (Z) sea 0.5 (mejora el resultado de regresión con Sigmoid)
                       repeat=1,             # Repite los elementos del dataset (sólo lo uso cuando overfitteo a un modelo y quiero usar batch_size > 1)
                       shuffle_pixels=False, # Mezcla aleatoriamente los pixeles dentro de cada modelo (no está implementado para edge_label y edge_index)
                       preload=True)         # Almacena el dataset en RAM

In [4]:
pixels, depth, edge_label, edge_index = dataset[0]

# Para graficar la nube de puntos
plot_3d_pointcloud(dataset.denormalize(pixels), depth, dataset.im_shape)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.008105…

In [5]:
# Para graficar las conexiones
plot_3d_pointcloud(dataset.denormalize(pixels), depth, dataset.im_shape, edge_index.transpose())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.008105…

In [6]:
print('pixels shape:', pixels.shape, '-> (XY, N)')
print('depth shape:', depth.shape, '-> (Z, N)')
print('edge_label shape:', edge_label.shape, '-> (N, k)')
print('edge_index shape:', edge_index.shape, '-> (2, N*k)')

pixels shape: (2, 2000) -> (XY, N)
depth shape: (2000,) -> (Z, N)
edge_label shape: (2000, 20) -> (N, k)
edge_index shape: (2, 40000) -> (2, N*k)


In [7]:
pixels

array([[-0.17777778, -0.24814814,  0.28148147, ...,  0.35555556,
         0.2851852 , -0.56296295],
       [ 0.05925926, -0.37407407, -0.02962963, ...,  0.04814815,
        -0.05185185,  0.04444445]], dtype=float32)

In [8]:
depth

array([0.3680138 , 0.50919026, 0.7052687 , ..., 0.47389615, 0.7052687 ,
       0.30919024], dtype=float32)

In [9]:
edge_label

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [10]:
edge_label.sum()

517

In [11]:
edge_index

array([[   0,    0,    0, ..., 1999, 1999, 1999],
       [ 227,  242,  296, ..., 1412, 1437, 1757]], dtype=int64)

In [12]:
N = pixels.shape[1]
r,c = edge_index
ady_matrix = np.zeros((N,N))
ady_matrix[r,c] = 1
for n in range(N):
    c = edge_index[1, edge_index[0,:] == n][edge_label[n, :] == 1]
    ady_matrix[n, c] = 2

In [13]:
# Esta matriz de adyacencia tiene 1 donde hay conexiones "buenas" y 2 donde hay conexiones que hay que eliminar
ady_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])